In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import torch
import pickle
from app.util.timer import Timer
from app.util.Differ import MatrixDiffer
from main import YoloRuntimeTest
import time
from threading import Thread

CPU input

In [2]:
args_pytorch = {
    "weights": "./app/weights/yolov9c.pt", 
    "source": "./app/assets/sample_image_1.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "inference_type": "ultralytics",
    "type": "image",
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

args_onnx = {
    "weights": "./app/weights/yolov9c-quantize.onnx", 
    "source": "./app/assets/sample_image_1.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "inference_type": "ultralytics",
    "type": "image", 
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

args_onnx_runtime_model = {
    "weights": "./app/weights/yolov9c-quantize.onnx", 
    "source": "./app/assets/sample_image_1.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "inference_type": "onnxruntime_model",
    "type": "image", 
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

args_openvino = {
    "weights": "./app/weights/yolov9c_openvino_model", 
    "source": "./app/assets/sample_image_1.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "inference_type": "ultralytics",
    "type": "image", 
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

args_openvino_model = {
    "weights": "./app/weights/yolov9c_openvino_model/yolov9c.xml", 
    "source": "./app/assets/sample_image_1.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "inference_type": "openvino_model",
    "type": "image", 
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

Initilize YOLO runtime test and Timer classes

In [3]:
yolo_runtime_test = YoloRuntimeTest()
timer = Timer()

In [ ]:
yolo_runtime_test.export_onnx_quantize()

In [ ]:
cpu_pytorch_ultralytics_image = yolo_runtime_test.ultralytics_run_image(args_pytorch)
print(cpu_pytorch_ultralytics_image)

In [ ]:
cpu_openvino_ultralytics_image = yolo_runtime_test.ultralytics_run_image(args_openvino)
print(cpu_openvino_ultralytics_image)

In [ ]:
cpu_openvino_model_image = yolo_runtime_test.openvino_run_image(args_openvino_model)
print(cpu_openvino_model_image)

In [4]:
cpu_onnx_ultralytics_image = yolo_runtime_test.ultralytics_run_image(args_onnx)
print(cpu_onnx_ultralytics_image)

[INFO] Inference Image
Loading app\weights\yolov9c-quantize.onnx for ONNX Runtime inference...

[[[     11.497      15.618      23.312 ...      543.81      569.74      603.58]
  [     22.046      21.142      21.218 ...      613.57      623.57      619.28]
  [     30.757      37.472      30.971 ...      405.02      423.79      386.74]
  ...
  [ 3.5763e-07  1.4901e-07  1.4901e-07 ...   2.563e-06  2.6226e-06  3.1292e-06]
  [ 3.2783e-07  2.9802e-07  2.6822e-07 ...  2.7716e-06  3.0994e-06  2.8014e-06]
  [ 1.7881e-07  1.1921e-07  2.0862e-07 ...  2.2352e-06  2.8908e-06  2.5034e-06]]]
0: 640x640 2 cars, 1 truck, 3 traffic lights, 184.5ms
Speed: 7.0ms preprocess, 184.5ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)
Elapsed time: 1.7478 seconds
([['traffic light', 0.8753820657730103, 513, 364, 564, 529], ['traffic light', 0.8653980493545532, 35, 342, 101, 514], ['car', 0.7800920605659485, 802, 1144, 898, 1232], ['car', 0.7567036151885986, 970, 1123, 1067, 1234], ['traffic lig

In [5]:
cpu_onnx_runtime_model_image = yolo_runtime_test.onnxruntime_run_image(args_onnx_runtime_model)
print(cpu_onnx_runtime_model_image)

[INFO] Inference Image
[[[     11.497      15.618      23.312 ...      543.81      569.74      603.58]
  [     22.046      21.142      21.218 ...      613.57      623.57      619.28]
  [     30.757      37.472      30.971 ...      405.02      423.79      386.74]
  ...
  [ 3.5763e-07  1.4901e-07  1.4901e-07 ...   2.563e-06  2.6226e-06  3.1292e-06]
  [ 3.2783e-07  2.9802e-07  2.6822e-07 ...  2.7716e-06  3.0994e-06  2.8014e-06]
  [ 1.7881e-07  1.1921e-07  2.0862e-07 ...  2.2352e-06  2.8908e-06  2.5034e-06]]]
Elapsed time: 0.1786 seconds
([['traffic light', 0.8753820657730103, 513, 464, 564, 587], ['traffic light', 0.8653980493545532, 35, 448, 101, 577], ['car', 0.7800920605659485, 802, 1050, 898, 1115], ['car', 0.7567036151885986, 970, 1034, 1066, 1117], ['traffic light', 0.6532491445541382, 956, 515, 1045, 599], ['truck', 0.6365572214126587, 424, 1016, 709, 1217]], 0.17863199999999857, 0.0)


Difference CPU

In [ ]:
pd.set_option('display.expand_frame_repr', False)

with open('pytorch_ultralytics_outputs.pkl', 'rb') as f:
    pytorch_ultralytics_outputs = pickle.load(f)
with open('openvino_ultralytics_outputs.pkl', 'rb') as f:
    openvino_ultralytics_outputs = pickle.load(f)
with open('onnx_ultralytics_outputs.pkl', 'rb') as f:
    onnx_ultralytics_outputs = pickle.load(f)
with open('onnx_model_outputs.pkl', 'rb') as f:
    onnx_model_outputs = pickle.load(f)

def generate_difference_df(output1, output2):
    differ = MatrixDiffer(output1, output2)
    result = differ.find_difference()
    return result

print(generate_difference_df(pytorch_ultralytics_outputs, openvino_ultralytics_outputs))
print(generate_difference_df(pytorch_ultralytics_outputs, onnx_ultralytics_outputs))
print(generate_difference_df(pytorch_ultralytics_outputs, onnx_model_outputs))
print(generate_difference_df(onnx_ultralytics_outputs, openvino_ultralytics_outputs))
print(generate_difference_df(onnx_ultralytics_outputs, onnx_model_outputs))
print(generate_difference_df(onnx_model_outputs, openvino_ultralytics_outputs))

Average CPU Time (10)

In [ ]:
result_time = []
ultralytics_inference_time = []

In [ ]:
def run_inference_with_args(inference_func, args):
    def wrapper():
        return inference_func(args)
    return wrapper

def collect_execution_times(run_inference_func, args, iterations=100):
    execution_times = []
    for i in range(iterations):
        args["source"] = f"./app/assets/sample_image_{i}.jpg"
        wrapper_func = run_inference_with_args(run_inference_func, args)
        result = wrapper_func()
        execution_times.append(result[1] * 1000)
        ultralytics_inference_time.append(result[2])
    return execution_times

args_pytorch = {
    "weights": "./app/weights/yolov9c.pt", 
    "source": "./app/assets/sample_image_0.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "inference_type": "ultralytics",
    "type": "image",
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

args_onnx = {
    "weights": "./app/weights/yolov9c.onnx", 
    "source": "./app/assets/sample_image_0.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "inference_type": "ultralytics",
    "type": "image", 
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

args_onnx_runtime_model = {
    "weights": "./app/weights/yolov9c.onnx", 
    "source": "./app/assets/sample_image_0.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "inference_type": "onnxruntime_model",
    "type": "image", 
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

args_openvino = {
    "weights": "./app/weights/yolov9c_openvino_model", 
    "source": "./app/assets/sample_image_0.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "inference_type": "ultralytics",
    "type": "image", 
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

args_openvino_model = {
    "weights": "./app/weights/yolov9c_openvino_model/yolov9c.xml", 
    "source": "./app/assets/sample_image_0.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "inference_type": "openvino_model",
    "type": "image", 
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

result_time.append(collect_execution_times(yolo_runtime_test.ultralytics_run_image, args_pytorch))
# result_time.append(collect_execution_times(yolo_runtime_test.ultralytics_run_image, args_openvino))
# result_time.append(collect_execution_times(yolo_runtime_test.openvino_run_image, args_openvino_model))
# result_time.append(collect_execution_times(yolo_runtime_test.ultralytics_run_image, args_onnx))
# result_time.append(collect_execution_times(yolo_runtime_test.onnxruntime_run_image, args_onnx_runtime_model))

In [ ]:
result_time.append(collect_execution_times(yolo_runtime_test.ultralytics_run_image, args_openvino))

In [ ]:
result_time.append(collect_execution_times(yolo_runtime_test.ultralytics_run_image, args_onnx))

In [ ]:
result_time.append(collect_execution_times(yolo_runtime_test.openvino_run_image, args_openvino_model))

In [ ]:
result_time.append(collect_execution_times(yolo_runtime_test.onnxruntime_run_image, args_onnx_runtime_model))

In [ ]:
result_time = np.array(result_time)
df = pd.DataFrame(np.transpose(result_time), 
                  columns=["pytorch+ultralytics time cpu (ms)",
                           "openvino+ultralytics time cpu (ms)",
                           "onnx​+ultralytics time cpu (ms)",
                           "openvino model time cpu (ms)", 
                           "onnx runtime model time cpu (ms)"])
df.describe(percentiles=[.9, .95])

In [ ]:
ultralytics_inference_time

In [ ]:
def split_list_into_arrays(input_list):
    list1 = input_list[:100]
    list2 = input_list[100:200]
    list3 = input_list[200:300]
    
    return list1, list2, list3

array1, array2, array3 = split_list_into_arrays(ultralytics_inference_time)
infer_timer_list = [array1, array2, array3]

print(infer_timer_list)

infer_timer_list = np.array(infer_timer_list)
df_infer = pd.DataFrame(np.transpose(infer_timer_list), columns=["pytorch+ultralytics built-in profiler time cpu (ms)", "openvino+ultralytics built-in profiler time cpu (ms)", "onnx+ultralytics built-in profiler time cpu (ms)"])
df_infer.describe(percentiles=[.9, .95])

Save CPU result

In [ ]:
with open('./app/saved_pkl/cpu_df.pkl', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
with open('./app/saved_pkl/cpu_infer_df.pkl', 'wb') as f:
    pickle.dump(df_infer, f)